# Arabic Digit Recognizer

In [ ]:
# Importing necessary libraries
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import backend as k
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import model_selection
import torch
from torch import nn, optim, utils
from torchvision import datasets, transforms

## Step 1: Process the data

### Image Processing

In [ ]:
# https://drive.google.com/file/d/1qnUw_wV1705XqEJ95wV8IF3GiM-yhMVj/view?usp=sharing
!gdown 1qnUw_wV1705XqEJ95wV8IF3GiM-yhMVj

In [ ]:
!unzip /content/arabic-dataset.zip

In [ ]:

df = pd.read_csv("/content/csvTrainImages 60k x 784.csv", header=None)


In [ ]:
train_labels =  pd.read_csv("/content/csvTrainLabel 60k x 1.csv", header=None)
test_labels = pd.read_csv("/content/csvTestLabel 10k x 1.csv", header=None)

In [ ]:
test_data = (pd.read_csv("/content/csvTestImages 10k x 784.csv", header=None))

In [ ]:
df.head()

In [ ]:
test_data = test_data.to_numpy()
train_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()
train_data = df.to_numpy()

In [ ]:
image = np.asarray(train_data[7,:]).squeeze().reshape([28,28])
plt.imshow(image)
plt.show()
print("The image above represent:")
print((train_labels[7]))

In [ ]:
image = np.asarray(test_data[8,:]).squeeze().reshape([28,28])
plt.imshow(image)
plt.show()
print("The image above represent:")
print((test_labels[8]))

In [ ]:
num_labels = 10
def reformat(labels):
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return labels

In [ ]:
train_labels = reformat(train_labels)
test_labels = reformat(test_labels)

In [ ]:
x_train = train_data
y_train = train_labels
x_test = test_data
y_test = test_labels

In [ ]:
y_train = y_train.reshape(x_train.shape[0],10)
y_test = y_test.reshape(x_test.shape[0],10)

In [ ]:
import cv2

In [ ]:
img = np.array(np.asarray(x_train[1,:]).squeeze().reshape([28,28]),np.uint8)

In [ ]:
plt.imshow(img)

In [ ]:
def ap_aug(img):
  img = np.array(np.asarray(img).squeeze().reshape([28,28]),np.uint8)
  img=cv2.resize(img,(28,28),interpolation=cv2.INTER_AREA)
  gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0)
  img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
  img = cv2.filter2D(img, -1, kernel)
  ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  return img

In [ ]:
plt.imshow(ap_aug(np.array(np.asarray(x_train[1,:]).squeeze().reshape([28,28]),np.uint8)))

In [ ]:
plt.imshow(ap_aug(x_train[9]))

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
X=[]
for i in range(len(x_train)):
  img = ap_aug(x_train[i,:])
  X.append(img)
X = np.asarray(X)

In [ ]:
Xt=[]
for i in range(len(x_test)):
  imgg = ap_aug(x_test[i,:])
  Xt.append(imgg)
Xt = np.asarray(Xt)

# Reshaping data to feed into the model

In [ ]:
x_train = X.reshape(x_train.shape[0],28,28,1) / 255.0
x_test = Xt.reshape(x_test.shape[0], 28, 28, 1) / 255.0

In [ ]:
x_train.shape, x_test.shape

Let's see some samples of the processed data. The digits are more sharp now

In [ ]:
plt.subplot(221)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[200], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[6150], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[5555], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

## Step 2: Design the model

In [ ]:
def my_model(img_size=28,channels=1):
    model = Sequential()
    input_shape = (img_size,img_size,channels)
    model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()

    return model
model=my_model()
model.summary()

## Step 3: Train the model

In [ ]:
path_model_64='arabic64-model_filter.h5' # save model at this location after each epoch

model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate
# fit the model
h=model.fit(x=x_train,
            y=y_train,
            batch_size=64,
            epochs=10,
            verbose=1,
            validation_data=(x_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model_64),
            ]
            )

In [ ]:
predictions_prob=model.predict(x_test) # get predictions for all the test data

In [ ]:
x_test.shape

In [ ]:
n_sample=200
np.random.seed(42)
ind=np.random.randint(0,len(x_test), size=n_sample)

In [ ]:
#Declaring constants
FIG_WIDTH=20 # Width of figure
HEIGHT_PER_ROW=3 # Height of each row when showing a figure which consists of multiple rows
RESIZE_DIM=28 # The images will be resized to 28x28 pixels

In [ ]:
def imshow_group(X,y,y_pred=None,n_per_row=10,phase='processed'):
    '''helper function to visualize a group of images along with their categorical true labels (y) and prediction probabilities.
    Args:
        X: images
        y: categorical true labels
        y_pred: predicted class probabilities
        n_per_row: number of images per row to be plotted
        phase: If the images are plotted after resizing, pass 'processed' to phase argument.
            It will plot the image and its true label. If the image is plotted after prediction
            phase, pass predicted class probabilities to y_pred and 'prediction' to the phase argument.
            It will plot the image, the true label, and it's top 3 predictions with highest probabilities.
    '''
    n_sample=len(X)
    img_dim=X.shape[1]
    j=int(np.ceil(n_sample/n_per_row))
    fig=plt.figure(figsize=(FIG_WIDTH,HEIGHT_PER_ROW*j))
    for i,img in enumerate(X):
        plt.subplot(j,n_per_row,i+1)
#         img_sq=np.squeeze(img,axis=2)
#         plt.imshow(img_sq,cmap='gray')
        plt.imshow(img)
        if phase=='processed':
            plt.title(np.argmax(y[i]))
        if phase=='prediction':
            top_n=3 # top 3 predictions with highest probabilities
            ind_sorted=np.argsort(y_pred[i])[::-1]
            h=img_dim+4
            for k in range(top_n):
                string='pred: {} ({:.0f}%)\n'.format(ind_sorted[k],y_pred[i,ind_sorted[k]]*100)
                plt.text(img_dim/2, h, string, horizontalalignment='center',verticalalignment='center')
                h+=4
            if y is not None:
                plt.text(img_dim/2, -4, 'true label: {}'.format(np.argmax(y[i])),
                         horizontalalignment='center',verticalalignment='center')
        plt.axis('off')
    plt.show()

In [ ]:
imshow_group(X=x_test[ind],y=None,y_pred=predictions_prob[ind], phase='prediction')

In [ ]:
def visualize_model(model, num_images=6, temp=0):
  for i in range(num_images):

    image = x_test[temp]
    plt.imshow(image)
    plt.show()
    print("The image above represent:")
    print(np.argmax(predictions_prob[temp]))
    temp = temp+4


In [ ]:
visualize_model(model)

In [ ]:
y_pred = predictions_prob

In [ ]:
test = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(predictions_prob,axis=1))

In [ ]:
confusion

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat=confusion, figsize=(8, 6), cmap=plt.cm.Greens)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

## Classification Report

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3','class 4','class 5','class 6','class 7','class 8','class 9']
print(classification_report(np.argmax(y_test,axis=1), np.argmax(predictions_prob,axis=1), target_names=target_names))

In [ ]:
import keras.utils as image

from google.colab import files

In [ ]:
# Model Prediction
print("========================== Model Prediction ==========================")
uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path)
  x = image.img_to_array(img)
  x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
  print("**************** Uploaded Image ****************")
  plt.imshow(x,cmap='gray')
  plt.show()
  x = cv2.resize(np.array(x), (28, 28))
  x = x.reshape(28,28,1)
  #x = ap_aug(x)
  x= x/255.0
  x = np.expand_dims(x, axis=0)

  class_label = model.predict(x.reshape(1,28,28,1))
  print(class_label)
  print("Predicted Value is:",np.where(class_label[0]==max(class_label[0]))[0])